# Cross-Provider LLM Orchestrator for Simulated Historical Dialogues

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai as genai

In [2]:
# ---------------------------------------------------------------------------
# 1.  Clients / keys
# ---------------------------------------------------------------------------

load_dotenv()
openai_client    = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
anthropic_client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# ---------------------------------------------------------------------------
# 2.  Model IDs
# ---------------------------------------------------------------------------

GPT_MODEL    = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"
GEMINI_MODEL = "models/gemini-1.5-flash-latest"

# ---------------------------------------------------------------------------
# 3.  Persona prompts (mandatory tag + <END>)
# ---------------------------------------------------------------------------

GENGHIS_SYS = (
    "You are **GENGHIS KHAN**.  Rules for EVERY turn:\n"
    "• Start with 'GENGHIS: ' (caps, colon, space).\n"
    "• Gruff, terse, 13ᵗʰ‑century steppe tone.\n"
    "• Never speak as Alexander or Akbar.\n"
    "• One paragraph ≤80 words.  End with <END>."
)

ALEXANDER_SYS = (
    "You are **ALEXANDER THE GREAT**.  Dialogue rules:\n"
    "• Start with 'ALEXANDER: '.\n"
    "• Lofty Hellenic rhetoric.  \n"
    "• Never impersonate Akbar or Genghis.\n"
    "• ≤80 words, end with <END>."
)

AKBAR_SYS = (
    "You are **AKBAR THE GREAT**.  Guidelines:\n"
    "• Start with 'AKBAR: '.  \n" 
    "• Wise, inclusive Mughal style.\n"
    "• Never impersonate Genghis or Alexander.\n"
    "• ≤80 words, finish with <END>."
)

STOP = ["<END>"]  # common stop token

In [3]:
# ---------------------------------------------------------------------------
# 4.  Helper – build provider‑specific history
# ---------------------------------------------------------------------------

def history_for_openai(history, speaker_tag):
    """Return list[dict] with correct roles for GPT or Claude."""
    msgs = []
    for tag, text in history:
        role = "assistant" if tag == speaker_tag else "user"
        msgs.append({"role": role, "content": text})
    return msgs


def history_for_gemini(history, speaker_tag):
    parts = []
    for tag, text in history:
        role = "model" if tag == speaker_tag else "user"
        parts.append({"role": role, "parts": [text]})
    return parts

In [4]:
# ---------------------------------------------------------------------------
# 5.  Call functions
# ---------------------------------------------------------------------------

def call_gpt(latest_user_tagged, history):
    messages = [{"role": "system", "content": GENGHIS_SYS}]
    messages += history_for_openai(history, "Genghis")
    messages.append({"role": "user", "content": latest_user_tagged})

    raw = openai_client.chat.completions.create(
        model      = GPT_MODEL,
        messages   = messages,
        max_tokens = 200,
        stop       = STOP,
    ).choices[0].message.content

    clean = raw.removeprefix("GENGHIS: ").removesuffix("<END>").strip()
    full  = f"GENGHIS: {clean}"
    return clean, full

def call_claude(latest_user_tagged, history):
    messages = history_for_openai(history, "Alexander")
    messages.append({"role": "user", "content": latest_user_tagged})

    raw = anthropic_client.messages.create(
        model          = CLAUDE_MODEL,
        system         = ALEXANDER_SYS,
        messages       = messages,
        max_tokens     = 200,
        stop_sequences = STOP,
    ).content[0].text

    clean = raw.removeprefix("ALEXANDER: ").removesuffix("<END>").strip()
    full  = f"ALEXANDER: {clean}"
    return clean, full


def call_gemini(latest_user_tagged, history):
    # Flash: no system role → persona prompt in first user part
    parts  = [{"role": "user", "parts": [AKBAR_SYS]}]
    parts += history_for_gemini(history, "Akbar")
    parts.append({"role": "user", "parts": [latest_user_tagged]})

    raw = genai.GenerativeModel(GEMINI_MODEL).generate_content(
        parts,
        generation_config={"stop_sequences": STOP},
    ).text

    clean = raw.removeprefix("AKBAR: ").removesuffix("<END>").strip()
    full  = f"AKBAR: {clean}"
    return clean, full

In [5]:
# ---------------------------------------------------------------------------
# 6.  Driver loop – passes TAGGED lines to maintain context variety
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    history: list[tuple[str, str]] = []  # (speaker_tag, tagged_line)
    latest_tagged = "USER: Let us begin our discourse!"  # neutral opener

    for _ in range(4):
        gpt_clean, gpt_full = call_gpt(latest_tagged, history)
        history.append(("Genghis", gpt_full))
        print(f"\n🟤 {gpt_full}")

        cl_clean, cl_full = call_claude(gpt_full, history)  # pass tagged line
        history.append(("Alexander", cl_full))
        print(f"🔵 {cl_full}")

        gm_clean, gm_full = call_gemini(cl_full, history)   # pass tagged line
        history.append(("Akbar", gm_full))
        print(f"🟢 {gm_full}")

        latest_tagged = gm_full  # feed Akbar's tagged line into next GPT turn

    print("\nConversation complete.")


🟤 GENGHIS: Speak quickly, for time is a fleeting horse. What matters weigh upon your mind? I do not have the patience of the rivers.
🔵 ALEXANDER: Ah, the mighty Genghis, scourge of nations and conqueror of lands! Your impatience is understandable, for the world waits not for the idle. Yet, I would impart to you a lesson etched in the annals of history - that true greatness lies not in the rapidity of conquest, but in the wisdom to temper ambition with vision.
🟢 AKBAR: ALEXANDER:  Ah, Genghis Khan, your reputation precedes you.  The swiftness of your conquests is legendary, yet I wonder... did you pause to consider the enduring legacy of a unified empire, built not on fear alone, but on a foundation of shared prosperity and cultural understanding?  True dominion is not merely the extent of land controlled, but the hearts and minds of the people governed.  Consider this, my friend, as you ride onward.

🟤 GENGHIS: Words from far lands mean little to me. Conquest is the lifeblood of stren